In [ ]:
# 사이트 : https://www.weather.go.kr/weather/main.jsp

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# 1. 동네예보에서 각 항목별 데이터 스크래핑
    - 기온
    - 체감온도
    - 바람
    - 습도
    - 강수확률
        => array or dataframe 구성

1) 최고 / 최저 기온 출력 

2) 체감온도 평균 출력  

3) 바람이 가장 많이 부는 방향은?  

4) '맑음'에 해당되는 날씨가 가장 오랜 기간동안 지속되는 시각을 출력  
    ex) 8월18일(화) 18시~8월19일(수) 24시

In [2]:
url = "https://www.weather.go.kr/weather/main.jsp"
driver = webdriver.Chrome("C:/mychrome/chromedriver.exe")
driver.get(url)

In [3]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [4]:
in_wid = soup.select("div.in-wid")
w_data = []
for wid in in_wid:
    d = 1
    for t in range(1, len(wid.select("ul:nth-child(3) > li"))):
        time = wid.select("ul:nth-child(3) > li:nth-child({})".format(t))[0].text
        date = wid.select("ul.m-tb.m-tb-date.bd-top.clearfix > li:nth-child({}) > span.bold".format(d))[0].text
        if int(time[:-1])==24:
            d += 1
        #기온
        temp = wid.select("g.highcharts-data-labels.highcharts-series-0.highcharts-line-series.highcharts-color-0 > g:nth-child({}) > text > tspan:nth-child(2)".format(t))[0].text
        #체감온도
        feel_temp = wid.select("ul:nth-child(6) > li:nth-child({})".format(t))[0].text[1:-1]
        #바람, 풍향
        wind = wid.select("ul:nth-child(7) > li:nth-child({}) > p".format(t))[0].text
        wind_dir = wid.select("ul:nth-child(7) > li:nth-child({}) > img".format(t))[0]['title']
        #습도
        wet = wid.select("ul:nth-child(8) > li:nth-child({}) > p".format(t))[0].text
        
        if t!=19:
            #강수확률
            pred_rain = wid.select("ul:nth-child(10) > li:nth-child({})".format(t))[0].text
            #날씨
            weather = wid.select("ul.m-tb.wid83.pd67.bd-no.img01.clearfix > li:nth-child({}) > img".format(t))[0]['title']
        
        w_data.append([date, time, weather, temp, feel_temp, wind, wind_dir, wet, pred_rain])

In [5]:
df = pd.DataFrame(w_data)
df.columns = ["날짜", '시간', '날씨', '기온', '체감온도', '바람', '바람방향', '습도', '강수확률']
df

,날짜,시간,날씨,기온,체감온도,바람,바람방향,습도,강수확률
0,8월 18일(화),21시,맑음,26℃,29℃,4km/h,서풍,90%,0%
1,8월 18일(화),24시,맑음,25℃,29℃,4km/h,서풍,95%,0%
2,8월 19일(수),3시,맑음,26℃,30℃,4km/h,서풍,95%,0%
3,8월 19일(수),6시,맑음,26℃,30℃,4km/h,남풍,95%,0%
4,8월 19일(수),9시,맑음,27℃,30℃,7km/h,남풍,80%,0%
5,8월 19일(수),12시,맑음,30℃,32℃,7km/h,남서풍,70%,0%
6,8월 19일(수),15시,맑음,32℃,33℃,7km/h,서풍,65%,0%
7,8월 19일(수),18시,맑음,28℃,30℃,7km/h,서풍,70%,0%
8,8월 19일(수),21시,맑음,26℃,29℃,4km/h,서풍,90%,0%
9,8월 19일(수),24시,맑음,25℃,29℃,4km/h,남서풍,95%,0%


In [6]:
temp_arr = df['기온'].values

In [7]:
feel_temp_arr = df['체감온도'].values
feel_temp_arr.tolist()
ft_list = []
for ft in feel_temp_arr:
    ft_list.append(int(ft[:-1]))
feel_temp_arr = np.array(ft_list)

In [8]:
winddir_arr = df['바람방향'].values
w, cnt = np.unique(winddir_arr, return_counts=True)
freq_wind = w[np.argmax(cnt)]

In [9]:
date_arr = df['날짜'].values
time_arr = df['시간'].values
cnt2, s_idx, e_idx = 0, 0, 0
max_date = []
for i, w in enumerate(df['날씨'].values):
    if w == '맑음':
        if cnt2 == 0 and s_idx == 0:
            s_idx = i
        cnt2 += 1
    else:
        if cnt2 > 0:
            e_idx = i-1
            max_date.append([cnt2, s_idx, e_idx])
            cnt2, s_idx, e_idx = 0, 0, 0
if cnt2 > 0:
    e_idx = i-1
    max_date.append([cnt2, s_idx, e_idx])

max_date.sort(key = lambda x : x[0], reverse=True)
s_idx, e_idx = max_date[0][1], max_date[0][2]
max_long_term = date_arr[s_idx]+" "+time_arr[s_idx]+" ~ "+date_arr[e_idx]+" "+time_arr[e_idx]

In [10]:
print("최고기온 : ", temp_arr.max())
print("최저기온 : ", temp_arr.min())
print("평균 체감 온도 : ", feel_temp_arr.mean())
print("바람이 가장 많이 부는 방향 : ", freq_wind)
print("\'맑음\'이 가장 오랜 기간 지속되는 시각 : ", max_long_term)

최고기온 :  32℃
최저기온 :  25℃
평균 체감 온도 :  30.294117647058822
바람이 가장 많이 부는 방향 :  서풍
'맑음'이 가장 오랜 기간 지속되는 시각 :  8월 18일(화) 21시 ~ 8월 20일(목) 18시


# 2. 중기예보에서 각 항목별 데이터 스크래핑

- 강수확률 평균

In [11]:
midterm = soup.select("div.tab02 > ul.m-tb.wid122.img02.bd-rt-bot.mt-6.clearfix")

In [12]:
pr2_li = []
for i, mid in enumerate(midterm):
    date2 = mid.select("li".format(i+1))
    for d in date2:
        sep = d.select("div")
        for k in range(len(sep)):
            pred_rain2 = d.select("div:nth-child({}) > p:nth-child(3)".format(k+1))[0].text[:-1]
            pr2_li.append(int(pred_rain2))

print("강수확률 평균 : ", np.array(pr2_li).mean())

강수확률 평균 :  43.84615384615385
